# 目的
#每日收盤後開始爬股價

#可以視情況條種是否抓營收，大概都隔月的10 後營收才會公布
## 【資料源】
來自 _BaseInfo  _CrawBase 會先去取基本相關資料 (股票號碼...等)

## 【輸出】
股票相關推薦圖表/數據

In [1]:
import pandas as pd
import threading
from datetime import datetime, timedelta

In [2]:
#%run _BaseInfoETF.ipynb
%run _CrawBase.ipynb
#%run _LineMsg.ipynb
%run _genHtml.ipynb
#%run RunRealTimeStock.ipynb

C:\Users\user\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [3]:
def MainStock(stock_number, start_month,end_time=datetime.now().strftime("%Y-%m-%d"),updateRevenue=False):  

    strmsg ='【stock_number】'+str(stock_number)+'\n' +'【start_month】'+str(start_month)+'\n'+'【end_time】'+str(end_time)+'\n' 
    try:
        #處理 Stock
        RowData_df_craw_stock,His_Stock ,isSuccess= craw_stock(stock_number,start_month,end_time)
        #data_final,df_final=DataCleaning(RowData_df_craw_stock)  
        
        #處理 Revenue
        if(updateRevenue):
            RowData_df_craw_revenue,His_Stock=craw_revenue(stock_number)
            df_Revenue=RevenueDataCleaning(RowData_df_craw_revenue)
            
        #處理相關指標轉成html 
        stock_data=data_process(RowData_df_craw_stock)
        level,interval_type, lower_bound, upper_bound=get_interval(stock_data.iloc[-1]['收盤價'],np.array(stock_data['收盤價']))
        stock_data['stock_number']=stock_number
        stock_data['level']=level
        stock_data['interval_type']=interval_type
        stock_data['lower_bound']=lower_bound
        stock_data['upper_bound']=upper_bound
        
        #print('sendLineMessage')
        stock_data['年月日'] = stock_data['年月日'].astype(str).str.replace('*', '', regex=False)
        stock_data['年月日'] = pd.to_datetime(stock_data['年月日'], format='%Y-%m-%d')
  
        save_to_df(stock_data.iloc[-1].to_frame().T)
        
        gen_html(stock_number,stock_data)
        #sendLineMessage('成功'+'\n'+strmsg )
    except Exception as error:
        #sendLineMessage('失败'+'\n'+strmsg)
        print('MainStock Fail'+'\n'+strmsg)
        print(f"An exception occurred with MainStock {stock_number}: {error}")

In [4]:
start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False

# 定義一個處理股票資料的函數，接收股票代號列表作為參數
def process_stock_codes(stock_codes):
    # 計算月份字串（僅需計算一次）
    strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
    for code in stock_codes:
        stock_number = str(code)
        try:
            MainStock(stock_number, start_month,datetime.now().strftime("%Y-%m-%d"), want_update_Revenue)
        except Exception as error:
            print(f"An exception occurred with stock {stock_number}: {error}")
            
def daily_craw_stock():
    
    # 分割資料，取前 500 個和其餘的股票代號
    first_500_codes = _baseInfo[_baseInfo['Type']=='上市'].dropna()['公司代號']
    remaining_codes =_baseInfo[_baseInfo['Type']!='上市'].dropna()['公司代號']

    # 創建兩個執行緒，分別處理前 500 個和其餘股票代號
    thread1 = threading.Thread(target=process_stock_codes, args=(first_500_codes,))
    thread2 = threading.Thread(target=process_stock_codes, args=(remaining_codes,))

    # 啟動執行緒
    thread1.start()
    thread2.start()

    # 等待兩個執行緒完成
    thread1.join()
    thread2.join()
    
    save_data_by_date(df_save_old_RunRealTimeStock,"Result/DailyRunRealTimeStock")
    save_data_by_date(df_save_df_to_excel,"Result/DailyFiles")

    print("Stock processing completed.")
    

In [ ]:
daily_craw_stock()

In [12]:
# 補資料 

today =datetime.now().strftime("%Y-%m-%d")
df_info=_baseInfo.copy()
fullpath = f'D:\\Project\\Jupyter\\Stock\\Main\\Result\\DailyRunRealTimeStock\\{today}.xlsx'
my=pd.read_excel(fullpath)

df_info['公司代號']=df_info['公司代號'].astype(str)
my['stock_number']=my['stock_number'].astype(str)
# 找出只在 df_info 中存在，但不在 my 中的 stock_number
aa=df_info[~df_info['公司代號'].isin(my['stock_number'])]
#stock_codes=list(set(aa.dropna()['公司代號']))




# 分割資料，取前 500 個和其餘的股票代號
first_500_codes = aa[aa['Type']=='上市'].dropna()['公司代號']
remaining_codes =aa[aa['Type']!='上市'].dropna()['公司代號']

# 創建兩個執行緒，分別處理前 500 個和其餘股票代號
thread1 = threading.Thread(target=process_stock_codes, args=(first_500_codes,))
thread2 = threading.Thread(target=process_stock_codes, args=(remaining_codes,))

# 啟動執行緒
thread1.start()
thread2.start()

# 等待兩個執行緒完成
thread1.join()
thread2.join()

save_data_by_date(df_save_old_RunRealTimeStock,"Result/DailyRunRealTimeStock")
save_data_by_date(df_save_df_to_excel,"Result/DailyFiles")

print("Stock processing completed.")

Stock_His 2358 2024-11-01 ===>外部抓取
2358
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=2358
Stock_His 1240 2024-11-01 ===>外部抓取
1240
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=1240&date=2024/11/01&id=&response=json
Stock_His 1259 2024-11-01 ===>外部抓取
1259
Stock_His 2443 2024-11-01 ===>外部抓取
2443
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=2443
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=1259&date=2024/11/01&id=&response=json
Stock_His 2462 2024-11-01 ===>外部抓取
2462
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=2462
Stock_His 1264 2024-11-01 ===>外部抓取
1264
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=1264&date=2024/11/01&id=&response=json
Stock_His 2464 2024-11-01 ===>外部抓取
2464
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=2464
Stock_His 1268 2024-11-01 ===>外部抓取
1268
Stock_His 24

https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=2065&date=2024/11/01&id=&response=json
Stock_His 3494 2024-11-01 ===>外部抓取
3494
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=3494
Stock_His 2066 2024-11-01 ===>外部抓取
2066
Stock_His 3533 2024-11-01 ===>外部抓取
3533
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=3533
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=2066&date=2024/11/01&id=&response=json
Stock_His 2067 2024-11-01 ===>外部抓取
2067
Stock_His 4989 2024-11-01 ===>外部抓取
4989
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=4989
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=2067&date=2024/11/01&id=&response=json
Stock_His 2070 2024-11-01 ===>外部抓取
2070
Stock_His 8101 2024-11-01 ===>外部抓取
8101
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8101
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradi

Stock_His 3118 2024-11-01 ===>外部抓取
3118
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3118&date=2024/11/01&id=&response=json
Stock_His 3122 2024-11-01 ===>外部抓取
3122
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3122&date=2024/11/01&id=&response=json
Stock_His 3128 2024-11-01 ===>外部抓取
3128
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3128&date=2024/11/01&id=&response=json
Stock_His 3131 2024-11-01 ===>外部抓取
3131
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3131&date=2024/11/01&id=&response=json
Stock_His 3141 2024-11-01 ===>外部抓取
3141
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3141&date=2024/11/01&id=&response=json
Stock_His 3147 2024-11-01 ===>外部抓取
3147
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3147&date=2024/11/01&id=&response=json
Stock_His 3152 2024-11-01 ===>外部抓取
3152
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3152&date=2024/11/01&id=&respon

Stock_His 3324 2024-11-01 ===>外部抓取
3324
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3324&date=2024/11/01&id=&response=json
Stock_His 3325 2024-11-01 ===>外部抓取
3325
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3325&date=2024/11/01&id=&response=json
Stock_His 3332 2024-11-01 ===>外部抓取
3332
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3332&date=2024/11/01&id=&response=json
Stock_His 3339 2024-11-01 ===>外部抓取
3339
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3339&date=2024/11/01&id=&response=json
Stock_His 3349 2024-11-01 ===>外部抓取
3349
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3349&date=2024/11/01&id=&response=json
Stock_His 3354 2024-11-01 ===>外部抓取
3354
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3354&date=2024/11/01&id=&response=json
Stock_His 3357 2024-11-01 ===>外部抓取
3357
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3357&date=2024/11/01&id=&respon

Stock_His 3564 2024-11-01 ===>外部抓取
3564
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3564&date=2024/11/01&id=&response=json
Stock_His 3567 2024-11-01 ===>外部抓取
3567
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3567&date=2024/11/01&id=&response=json
Stock_His 3570 2024-11-01 ===>外部抓取
3570
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3570&date=2024/11/01&id=&response=json
Stock_His 3577 2024-11-01 ===>外部抓取
3577
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3577&date=2024/11/01&id=&response=json
Stock_His 3580 2024-11-01 ===>外部抓取
3580
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3580&date=2024/11/01&id=&response=json
Stock_His 3581 2024-11-01 ===>外部抓取
3581
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3581&date=2024/11/01&id=&response=json
Stock_His 3587 2024-11-01 ===>外部抓取
3587
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3587&date=2024/11/01&id=&respon

Stock_His 4153 2024-11-01 ===>外部抓取
4153
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4153&date=2024/11/01&id=&response=json
Stock_His 4154 2024-11-01 ===>外部抓取
4154
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4154&date=2024/11/01&id=&response=json
Stock_His 4157 2024-11-01 ===>外部抓取
4157
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4157&date=2024/11/01&id=&response=json
Stock_His 4160 2024-11-01 ===>外部抓取
4160
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4160&date=2024/11/01&id=&response=json
Stock_His 4161 2024-11-01 ===>外部抓取
4161
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4161&date=2024/11/01&id=&response=json
Stock_His 4162 2024-11-01 ===>外部抓取
4162
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4162&date=2024/11/01&id=&response=json
Stock_His 4163 2024-11-01 ===>外部抓取
4163
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4163&date=2024/11/01&id=&respon

Stock_His 4563 2024-11-01 ===>外部抓取
4563
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4563&date=2024/11/01&id=&response=json
Stock_His 4568 2024-11-01 ===>外部抓取
4568
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4568&date=2024/11/01&id=&response=json
Stock_His 4577 2024-11-01 ===>外部抓取
4577
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4577&date=2024/11/01&id=&response=json
Stock_His 4580 2024-11-01 ===>外部抓取
4580
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4580&date=2024/11/01&id=&response=json
Stock_His 4584 2024-11-01 ===>外部抓取
4584
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4584&date=2024/11/01&id=&response=json
Stock_His 4609 2024-11-01 ===>外部抓取
4609
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4609&date=2024/11/01&id=&response=json
Stock_His 4702 2024-11-01 ===>外部抓取
4702
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=4702&date=2024/11/01&id=&respon

Stock_His 5014 2024-11-01 ===>外部抓取
5014
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5014&date=2024/11/01&id=&response=json
Stock_His 5015 2024-11-01 ===>外部抓取
5015
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5015&date=2024/11/01&id=&response=json
Stock_His 5016 2024-11-01 ===>外部抓取
5016
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5016&date=2024/11/01&id=&response=json
Stock_His 5201 2024-11-01 ===>外部抓取
5201
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5201&date=2024/11/01&id=&response=json
Stock_His 5202 2024-11-01 ===>外部抓取
5202
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5202&date=2024/11/01&id=&response=json
Stock_His 5205 2024-11-01 ===>外部抓取
5205
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5205&date=2024/11/01&id=&response=json
Stock_His 5206 2024-11-01 ===>外部抓取
5206
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5206&date=2024/11/01&id=&respon

Stock_His 5426 2024-11-01 ===>外部抓取
5426
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5426&date=2024/11/01&id=&response=json
Stock_His 5432 2024-11-01 ===>外部抓取
5432
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5432&date=2024/11/01&id=&response=json
Stock_His 5438 2024-11-01 ===>外部抓取
5438
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5438&date=2024/11/01&id=&response=json
Stock_His 5439 2024-11-01 ===>外部抓取
5439
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5439&date=2024/11/01&id=&response=json
Stock_His 5443 2024-11-01 ===>外部抓取
5443
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5443&date=2024/11/01&id=&response=json
Stock_His 5450 2024-11-01 ===>外部抓取
5450
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5450&date=2024/11/01&id=&response=json
Stock_His 5452 2024-11-01 ===>外部抓取
5452
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=5452&date=2024/11/01&id=&respon

Stock_His 6104 2024-11-01 ===>外部抓取
6104
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=6104&date=2024/11/01&id=&response=json
Stock_His 6109 2024-11-01 ===>外部抓取
6109
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=6109&date=2024/11/01&id=&response=json
Stock_His 6111 2024-11-01 ===>外部抓取
6111
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=6111&date=2024/11/01&id=&response=json
Stock_His 6113 2024-11-01 ===>外部抓取
6113
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=6113&date=2024/11/01&id=&response=json
Stock_His 6114 2024-11-01 ===>外部抓取
6114
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=6114&date=2024/11/01&id=&response=json
Stock_His 6118 2024-11-01 ===>外部抓取
6118
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=6118&date=2024/11/01&id=&response=json
Stock_His 6121 2024-11-01 ===>外部抓取
6121
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=6121&date=2024/11/01&id=&respon